# Validación, Normalización y Match de Zonas en CSV

Este notebook permite validar, limpiar y normalizar los datos de origen y destino de un archivo CSV, y realizar el match correcto con la zona correspondiente. El objetivo es asegurar que los datos estén listos para ser importados al backend sin errores de codificación ni de matching.

---

## 1. Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import os

# Opcional: para warnings y display
import warnings
warnings.filterwarnings('ignore')

# Configuración de display
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1200)

In [ ]:
# Ruta del archivo CSV a validar
csv_path = r'C:\Users\ferna\Downloads\este es el que hay que subir.csv'

# Intentar cargar el archivo con encoding utf-8, fallback a latin1 si falla
try:
    df = pd.read_csv(csv_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, encoding='latin1')

print(f"Archivo cargado: {csv_path}")

In [ ]:
# Visualizar las primeras filas para inspeccionar la estructura y los datos
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head(10)

In [ ]:
# Validar estructura y contenido del archivo
required_columns = ['origen', 'destino', 'zona']
missing_cols = [col for col in required_columns if col not in df.columns]

if missing_cols:
    print(f"Faltan columnas requeridas: {missing_cols}")
else:
    print("Todas las columnas requeridas están presentes.")

# Revisar nulos y valores únicos
print("\nValores nulos por columna:")
print(df[required_columns].isnull().sum())

print("\nValores únicos por columna:")
for col in required_columns:
    print(f"{col}: {df[col].nunique()} únicos")

In [ ]:
# Función de normalización robusta
import re
def normalize_str(s):
    if pd.isnull(s):
        return ''
    s = str(s).strip().lower()
    s = s.replace('ñ', 'n')
    s = unicodedata.normalize('NFD', s)
    s = ''.join(c for c in s if unicodedata.category(c) != 'Mn')
    s = re.sub(r'\s+', ' ', s)
    return s

# Aplicar normalización a columnas de origen y destino
df['origen_norm'] = df['origen'].apply(normalize_str)
df['destino_norm'] = df['destino'].apply(normalize_str)

print("Ejemplo de normalización:")
df[['origen', 'origen_norm', 'destino', 'destino_norm']].head(10)

In [ ]:
# Crear catálogo de zonas normalizadas
zonas_catalogo = df[['zona']].drop_duplicates().copy()
zonas_catalogo['zona_norm'] = zonas_catalogo['zona'].apply(normalize_str)

# Diccionario para match rápido
zona_map = dict(zip(zonas_catalogo['zona_norm'], zonas_catalogo['zona']))

# Normalizar zona en el dataframe principal
df['zona_norm'] = df['zona'].apply(normalize_str)

# Asignar zona original a cada fila según zona normalizada (puede usarse para validación cruzada)
df['zona_match'] = df['zona_norm'].map(zona_map)

# Mostrar posibles problemas de matching
problemas = df[df['zona_match'].isnull() | (df['zona'] != df['zona_match'])]
if not problemas.empty:
    print('Filas con problemas de matching de zona:')
    display(problemas[['origen', 'destino', 'zona', 'zona_norm', 'zona_match']].head(20))
else:
    print('Todas las zonas fueron correctamente normalizadas y matcheadas.')

In [ ]:
# Mostrar resultados finales: origen/destino/zona normalizados y zona asignada
cols = ['origen', 'origen_norm', 'destino', 'destino_norm', 'zona', 'zona_norm', 'zona_match']
df[cols].head(20)